In [ ]:
!pip install pandas pyarrow pdf2image

In [ ]:
import pandas as pd
import pyarrow
import pdf2image
from PIL import Image

In [ ]:
#!rm -f *.jpg

In [ ]:
#!tar -xvf ../echantillon/echantillon_pdf.tar

In [ ]:
import os
df = pd.DataFrame(columns=["num_dossier","img","index"],dtype=object)
DIR_DOCX="./"
num_parquet=0
for i, num_dossier in enumerate(
    {file_name[:12] for file_name in os.listdir(DIR_DOCX) if file_name.endswith(".pdf")}
):


    pdf_file=[
            DIR_DOCX+file_name
            for file_name in os.listdir(DIR_DOCX)
            if file_name.endswith(".pdf") and file_name.startswith(num_dossier)
        ][0]

    images=pdf2image.convert_from_path(pdf_path=pdf_file,thread_count=os.cpu_count())
    for i,image in enumerate(images):
        image.save(num_dossier+"-"+str(i+1)+".jpg")


In [ ]:
#!rm -f echantillon_jpeg_allpages.tar

In [ ]:
#!tar -cvf   echantillon_jpeg_allpages.tar *.jpg

In [ ]:
import os
df = pd.DataFrame(columns=["num_dossier","path_img","num_page","num_max_page","index"],dtype=object)
DIR_DOCX="./"
num_parquet=0
for i, num_dossier in enumerate(
    {file_name[:12] for file_name in os.listdir(DIR_DOCX) if file_name.endswith(".jpg")}
):


    jpg_file=[
            DIR_DOCX+file_name
            for file_name in os.listdir(DIR_DOCX)
            if file_name.endswith(".jpg") and file_name.startswith(num_dossier)
        ]
    num_max_page=len(jpg_file)
    for file in jpg_file:
        num_page=(file.split("-")[1]).split(".")[0]
        
            
        try:
            new_line = {
                "num_dossier": num_dossier,
                "path_img": file,
                "num_page": num_page,
                "num_max_page": num_max_page,
                "index" : [i]
            }
        except:
            print(f"bug {num_dossier}")
            
        df_new_row = pd.DataFrame.from_records(data=new_line)
        df = pd.concat([df, df_new_row])

In [ ]:
df=df.drop(columns="index")
df=df.set_index(["num_dossier","num_page"])

In [ ]:
df.to_parquet("Dares_1Kaccords_dataset_image_metadata.parquet")